In [107]:
import re
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
node = []
allDatas = []
siganlList = []
SignalsName = []
messageName = []

In [2]:
def readFile(file_name):
    global node,allDatas,siganlList,SignalsName,messageName
    ''' 得到dbc文件的绝对路径'''
    filePath = file_name
    if filePath:
        print(filePath)
        f = open(filePath, "r")  # 设置文件对象
    else:
        print("读取文件失败！")
        return 0
    """
    NodesPattern:节点
    MessagePattern：消息
    SignalPattern：信号
    """
    NodesPattern = re.compile(r"BU_: (.*)", re.S)
    MessagePattern = re.compile(r"BO_ (.*?) (.*?): (.*?) (.*)", re.S)
#     SignalPattern = re.compile('''SG_ (.*?) : (.*?)\|(.*?)@.*? \((.*?),(.*?)\) \[(.*?)\|(.*?)\] "(.*?)" (.*)''', re.S)
    SignalPattern = re.compile('''SG_ (.*?) : (.*?)\|(.*?)@([0-9])([+|-]) \((.*?),(.*?)\) \[(.*?)\|(.*?)\] "(.*?)" (.*)''', re.S)
    DefaultValue = '''BA_ "GenSig(.*?)" SG_ (\d+) signalname (\d+);'''

    line = f.readline()
    allDatas=[]
    while line:
        """ 匹配出节点 """
        NodesSearched = re.search(NodesPattern, line.strip())
        if NodesSearched:
            node = NodesSearched.group(1).split(" ")
            #print(node)
        """ 匹配出消息 """
        MessageSearched = re.search(MessagePattern, line.strip())
        if MessageSearched:
            siganlList.clear()
            """如果匹配到了message，则获取到message的相关参数 
             比如匹配到了NM_Message_ESC_409，则会解析出改message的一些参数构成list对象['1033', 'NM_Message_ESC_409', '8', 'ESC']
             这四个参数分别是 messgage ID ;message name ; messgae dataLenth ,message sender
             而且把这个list对象 加在了 siganlList 索引0的位置
            """
            Message = list(MessageSearched.groups())
            siganlList.append(Message)
            """ 只 要 message的名字 messageName 列表中"""
            messageName.append(Message[1])
            """读取下一行"""
            line = f.readline()
            """因为有些message并没有定义signal，所以 下一行还是message"""
            MessageSearched = re.search(MessagePattern, line.strip())
            SignalSearched = re.search(SignalPattern, line.strip())
            """下一行如果不是message的内容 就一定是signal的内容了"""
            if not MessageSearched:
                while SignalSearched:
                    """获取信号的参数追加到siganlList"""
                    signal = list(SignalSearched.groups())
                    siganlList.append(signal)
                    """只获取 signal name"""
                    SignalsName.append(signal[0])

                    # 再次解析信号，直到这个message下的信号全部解析完毕
                    line = f.readline()
                    SignalSearched = re.search(SignalPattern, line.strip())
           # print(siganlList)
            c = copy.deepcopy(siganlList)
            allDatas.append(c)
        else:
            line = f.readline()
            MessageSearched = re.search(MessagePattern, line.strip())
    f.close()  # 将文件关闭
    return 1

In [108]:
import os
file_names = []
# path = os.path.abspath('../My_DBC')
path = os.path.abspath('../opendbc-master/opendbc-master-delFalseData')
# path = os.path.abspath('./opendbc-master/opendbc-master')
dirs = os.listdir(path)                    # 获取指定路径下的文件
for i in dirs:
    if os.path.splitext(i)[1] == ".dbc":
        file_names.append(os.path.join(path,i))
# file_names

In [4]:
def get_id_labels(index):
    file_name = file_names[index]
    readFile(file_name)

    # 将DBC文件中的ID(10进制数)改为ID(16进制数)
    data_gts = {}
    for alldata in allDatas:
        data_gts[str(hex(int(alldata[0][0])))[2:]] = alldata

    # 得到DBC文件中各ID Message对应信号位置
    all_id_lsbs = {}
    for k in data_gts.keys():
        message = data_gts[k]
        lsb_list = []
        message_id = message[0][0]+"_"+message[0][1]
        del message[0]
        for signal in message:
            start = int(signal[1])
            length = int(signal[2])
            endianness = int(signal[3])
            start_row = int(start / 8)
            start_col = 8 - (start - start_row * 8) - 1
            new_start = start_row * 8 + start_col
            if endianness == 0:
                new_end = new_start + length
                cur_lsb = new_end - 1
            if endianness == 1:
                cur_lsb = new_start
            lsb_list.append(cur_lsb)
        all_id_lsbs[k] = lsb_list
    return all_id_lsbs

In [5]:
def get_signal_positions(index):
    file_name = file_names[index]
    readFile(file_name)
    
    data_gts = {}
    for alldata in allDatas:
        data_gts[str(hex(int(alldata[0][0])))[2:]] = alldata
        
    signal_positions = {}
    flag = False
    for k in data_gts.keys():
        message = data_gts[k]
        matrix = [[0] * 8 for _ in range(8)]
        message_id = message[0][0]+"_"+message[0][1]
        del message[0]
        cur_signal_positions = []
        for signal in message:
            start = int(signal[1])
            length = int(signal[2])
            endianness = int(signal[3])
            start_row = int(start / 8)
            start_col = 8 - (start - start_row * 8) - 1
            new_start = start_row * 8 + start_col
            if endianness == 0:
                new_end = new_start + length
                cur_msb = new_start
                cur_lsb = new_end - 1
            if endianness == 1:
                cur_lsb = new_start
                count = 0
                x = start_row
                y = start_col
                while count < (length-1):
                    if y == 0:
                        y = 7
                        x = x + 1
                    else:
                        y = y - 1
                    count = count + 1
                cur_msb = x * 8 + y
            cur_signal_positions.append((cur_msb, cur_lsb))
        signal_positions[k] = cur_signal_positions
    return signal_positions

In [6]:
def get_label01(all_id_lsbs):
    # 将各ID Message的信号位置改为 0/1 label
    all_id_labels = {}
    for k in all_id_lsbs:
        cur_lsb_list = all_id_lsbs[k]
        cur_label = np.zeros(64)
        for cur_lsb in cur_lsb_list:
            cur_label[cur_lsb] = 1
        all_id_labels[k] = cur_label
    return all_id_labels

In [7]:
def get_labels(all_id_lsbs):
    # 将各ID Message的信号位置改为 0/1 label
    all_id_labels = {}
    for k in all_id_lsbs:
        cur_lsb_list = all_id_lsbs[k]
        cur_label = [2] * 64
        for cur_lsb in cur_lsb_list:
            cur_label[cur_lsb] = 1
        all_id_labels[k] = cur_label
    return all_id_labels

In [8]:
def get_trace(data):
    group = data.groupby(by="ID")
    id_tracedict = {}
    for trace in list(group):
        id_tracedict[trace[0]] = trace[1]["bin"].apply(lambda x: list(x)).apply(lambda x: list(map(int, x))).tolist()
    return id_tracedict

In [9]:
def get_ids(id_tracedict_int, all_id_labels):
    ids = []
    for id in id_tracedict_int.keys():
        if(id_tracedict_int.__contains__(id) and all_id_labels.__contains__(id)):
            ids.append(id)
    return ids

In [10]:
import random
# 设置全局的随机种子
random.seed(100)
# 以id为单位shuffle
def get_train_test_ids(id_tracedict, all_id_labels):
    ids = get_ids(id_tracedict, all_id_labels)
    random.shuffle(ids)
    train_len = int(len(ids)*0.9)
    train_ids = ids[:train_len]
    test_ids = ids[train_len:]
    return train_ids, test_ids

In [11]:
def get_trace_data(id_tracedict_int, all_id_labels): 
    cur_trace_data = []
    cur_trace_label = []
    for id in id_tracedict_int.keys():
        cur_trace = id_tracedict_int[id]
        cur_label = all_id_labels[id]
        length = len(cur_trace)
        start = 4000
        end = start + 1000
        n = int((length-4000)/1000)
        for i in range(n):
            cur_trace_data.append(cur_trace[start: end])
            cur_trace_label.append(cur_label)
            start = end
            end = start + 1000
    return cur_trace_data, cur_trace_label

In [12]:
def get_data_label(id_tracedict_int, all_id_labels):
    cur_trace_data, cur_trace_label = get_trace_data(id_tracedict_int, all_id_labels)
    trace_indexs = pd.DataFrame(cur_trace_label).index.tolist()
    random.shuffle(trace_indexs)
    trace_data, trace_labels = np.array(cur_trace_data)[trace_indexs], np.array(cur_trace_label)[trace_indexs]
    return trace_data, trace_labels

In [13]:
def get_trace_data_dict(cur_ids, id_tracedict_int, all_id_labels, trace_data_dict, trace_label_dict): 
    for id in cur_ids:
        trace_data_dict[id] = id_tracedict_int[id]
        trace_label_dict[id] = all_id_labels[id]

In [109]:
honda_dbcs = []
cadillac_dbcs = []
tesla_dbcs = []
gm_dbcs = []
for i in range(len(file_names)):
    cur_name = file_names[i].split("\\")[-1].split("_")[0]
    if cur_name == "honda":
        honda_dbcs.append(i)
    if cur_name == "cadillac":
        cadillac_dbcs.append(i)
    if cur_name == "tesla":
        tesla_dbcs.append(i)
    if cur_name == "gm":
        gm_dbcs.append(i)

In [110]:
honda_id_labels = {}
for cur_dbc_index in honda_dbcs:
    honda_id_labels.update(get_label01(get_id_labels(cur_dbc_index))) 
print(len(honda_id_labels)) 

cadillac_id_labels = {}
for cur_dbc_index in cadillac_dbcs:
    cadillac_id_labels.update(get_label01(get_id_labels(cur_dbc_index)))  
for cur_dbc_index in gm_dbcs:
    cadillac_id_labels.update(get_label01(get_id_labels(cur_dbc_index))) 
print(len(cadillac_id_labels)) 

# chevy_id_labels = {}
# for cur_dbc_index in gm_dbcs:
#     chevy_id_labels.update(get_label01(get_id_labels(cur_dbc_index))) 
# print(len(chevy_id_labels)) 

D:\--storage--\program\Code\jupyter-notebook\CarNetworkIDS\Code\MyTask\opendbc-master\opendbc-master-delFalseData\honda_accord_2018_can_generated.dbc
D:\--storage--\program\Code\jupyter-notebook\CarNetworkIDS\Code\MyTask\opendbc-master\opendbc-master-delFalseData\honda_civic_hatchback_ex_2017_can_generated.dbc
D:\--storage--\program\Code\jupyter-notebook\CarNetworkIDS\Code\MyTask\opendbc-master\opendbc-master-delFalseData\honda_civic_sedan_16_diesel_2019_can_generated.dbc
D:\--storage--\program\Code\jupyter-notebook\CarNetworkIDS\Code\MyTask\opendbc-master\opendbc-master-delFalseData\honda_civic_touring_2016_can_generated.dbc
D:\--storage--\program\Code\jupyter-notebook\CarNetworkIDS\Code\MyTask\opendbc-master\opendbc-master-delFalseData\honda_clarity_hybrid_2018_can_generated.dbc
D:\--storage--\program\Code\jupyter-notebook\CarNetworkIDS\Code\MyTask\opendbc-master\opendbc-master-delFalseData\honda_crv_executive_2016_can_generated.dbc
D:\--storage--\program\Code\jupyter-notebook\CarNet

In [16]:
# 得到Honda车的子数据集数据
data_hd = pd.read_csv('../Data/Honda/honda001.csv')
data_hd["bin"] = data_hd["Data"].apply(int, base=16).apply(bin).str[2:].apply(lambda x: x.zfill(64))

# 得到Honda车子的数据集的大Trace dict
id_tracedict_hd = get_trace(data_hd)

# 得到用于训练和测试的id
train_ids_hd, test_ids_hd = get_train_test_ids(id_tracedict_hd, honda_id_labels)

In [17]:
# 得到Cadillac车的子数据集数据
data_cd = pd.read_csv('../Data/Cadillac/CadillacCSV/001.txt.csv')
data_cd["bin"] = data_cd["Data"].apply(int, base=16).apply(bin).str[2:].apply(lambda x: x.zfill(64))

# 得到Honda车子的数据集的大Trace dict
id_tracedict_cd = get_trace(data_cd)

# 得到用于训练和测试的id
train_ids_cd, test_ids_cd = get_train_test_ids(id_tracedict_cd, cadillac_id_labels)

In [18]:
# 得到honda和caddilac车的trace dict及对应的label dict
train_data_dict = {}
train_label_dict = {}
get_trace_data_dict(train_ids_hd, id_tracedict_hd, honda_id_labels, train_data_dict, train_label_dict)
get_trace_data_dict(train_ids_cd, id_tracedict_cd, cadillac_id_labels, train_data_dict, train_label_dict)
# get_trace_data_dict(train_ids_ch, id_tracedict_ch, chevy_id_labels, train_data_dict, train_label_dict)

test_data_dict = {}
test_label_dict = {}
get_trace_data_dict(test_ids_hd, id_tracedict_hd, honda_id_labels, test_data_dict, test_label_dict)
get_trace_data_dict(test_ids_cd, id_tracedict_cd, cadillac_id_labels, test_data_dict, test_label_dict)
# get_trace_data_dict(test_ids_ch, id_tracedict_cd, cadillac_id_labels, test_data_dict, test_label_dict)

In [19]:
# 使用READ方法计算bit-flip
def get_bit_Flips(cur_id_traces):
    bit_Flips = []
    for i in range(len(cur_id_traces)):
        messagelist = cur_id_traces[i]
        trace_len = len(messagelist)
        bit_flip = np.zeros(64)
        previous = messagelist[0]
        for item in messagelist:
            for ix in range(64):
                if item[ix] != previous[ix]:
                    bit_flip[ix] = bit_flip[ix] + 1
            previous = item
        for ix in range(64):
            bit_flip[ix] = bit_flip[ix] / trace_len
        bit_flip = np.append(bit_flip, 0)
        bit_flip = np.append(bit_flip, 0)
        bit_Flips.append(bit_flip)
    return bit_Flips

In [20]:
# 先删除constant bit，然后，将trace序列拼成64。问题：会破环bit之间的相关性
def del_constant_bit_concate(cur_trace_data, cur_trace_label, bit_Flips):
    all_trace_data = []
    all_trace_label = []
    for i in range(len(bit_Flips)):
        cur_bit_flip = bit_Flips[i]
        cur_constant_bits = []
        for j in range(64):
            if cur_bit_flip[j] == 0:
                cur_constant_bits.append(j)
        cur_trace = []
        for temp in cur_trace_data[i]:
            cur_trace.append([temp[j] for j in range(len(temp)) if j not in cur_constant_bits])
        cur_label = []
        for j in range(len(cur_trace_label[i])):
            next_j = j+1
            if j not in cur_constant_bits:
                if next_j in cur_constant_bits:
                    cur_label.append(1)
                else:
                    cur_label.append(cur_trace_label[i][j])   
        if i == 0:
            all_trace_data = np.array(cur_trace)
            all_trace_label = np.array(cur_label)
        else:
            all_trace_data = np.hstack((all_trace_data, np.array(cur_trace)))
            all_trace_label = np.hstack((all_trace_label, np.array(cur_label)))
    
    
    num = int(all_trace_data.shape[1]/64)
    cur_trace_data = np.array(np.array_split(all_trace_data[:,0:num*64], num, axis=1))
    cur_trace_label = np.array(np.array_split(all_trace_label[0:num*64], num, axis=0))
    return cur_trace_data, cur_trace_label

In [21]:
def fill_list(cur_list, number, length):
    for i in range(len(cur_list),length):
        cur_list.append(number)
    return cur_list

In [22]:
# 只删除constant bit，形成不定长的trace 序列
def del_constant_bit(cur_trace_data, cur_trace_label, bit_Flips):
    all_trace_data = []
    all_trace_label = []
    for i in range(len(bit_Flips)):
        cur_bit_flip = bit_Flips[i]
        cur_constant_bits = []
        for j in range(64):
            if cur_bit_flip[j] == 0:
                cur_constant_bits.append(j)
        cur_trace = []
        for temp in cur_trace_data[i]:
            res_trace = [temp[j] for j in range(len(temp)) if j not in cur_constant_bits]
            cur_trace.append(fill_list(res_trace, -1, 64))
        cur_label = []
        for j in range(len(cur_trace_label[i])):
            next_j = j+1
            if j not in cur_constant_bits:
                if next_j in cur_constant_bits:
                    cur_label.append(1)
                else:
                    cur_label.append(cur_trace_label[i][j])  
        cur_label = fill_list(cur_label, -1, 64)
        all_trace_data.append(cur_trace)
        all_trace_label.append(cur_label)
    return np.array(all_trace_data), np.array(all_trace_label)

In [23]:
# 只删除constant bit，形成不定长的trace 序列
def del_constant_bit_new(cur_trace_data, cur_trace_label, bit_Flips):
    all_trace_data = []
    all_trace_label = []
    for i in range(len(bit_Flips)):
        cur_bit_flip = bit_Flips[i]
        cur_constant_bits = []
        for j in range(64):
            if cur_bit_flip[j] == 0:
                if j == 0 or cur_bit_flip[j-1] == 0:
                    cur_constant_bits.append(j)
        cur_trace = []
        for temp in cur_trace_data[i]:
            res_trace = [temp[j] for j in range(len(temp)) if j not in cur_constant_bits]
            cur_trace.append(fill_list(res_trace, -1, 64))
        cur_label = []
        for j in range(len(cur_trace_label[i])):
            next_j = j+1
            if j not in cur_constant_bits:
                if cur_bit_flip[j] != 0 and cur_bit_flip[next_j] == 0:
                    cur_label.append(1)
                else:
                    cur_label.append(cur_trace_label[i][j])  
        cur_label = fill_list(cur_label, -1, 64)
        all_trace_data.append(cur_trace)
        all_trace_label.append(cur_label)
    return np.array(all_trace_data), np.array(all_trace_label)

In [24]:
def change_labels(cur_trace_data, cur_trace_label, bit_Flips):
    for i in range(len(bit_Flips)):
        cur_bit_flip = bit_Flips[i]
        for j in range(64):
            if cur_bit_flip[j] == 0:
                cur_trace_label[i][j] = 0
    return cur_trace_data, cur_trace_label

In [25]:
# shuffle后得到训练集和测试集，其中训练集的id占70%，测试集的id占30%
train_data, train_labels = get_data_label(train_data_dict, train_label_dict)
bit_Flips_train = get_bit_Flips(train_data)

In [26]:
# train_data, train_labels = change_labels(train_data, train_labels, bit_Flips_train)
train_data_del, train_labels_del = del_constant_bit(train_data, train_labels, bit_Flips_train)
# train_data_del_new, train_labels_del_new = del_constant_bit_new(train_data, train_labels, bit_Flips_train)

In [27]:
test_data, test_labels = get_data_label(test_data_dict, test_label_dict)
bit_Flips_test = get_bit_Flips(test_data)

In [28]:
# test_data, test_labels = change_labels(test_data, test_labels, bit_Flips_test)
test_data_del, test_labels_del = del_constant_bit(test_data, test_labels, bit_Flips_test)
# test_data_del_new, test_labels_del_new = del_constant_bit_new(test_data, test_labels, bit_Flips_test)

In [29]:
train_data_lstm = train_data.transpose(0,2,1)
test_data_lstm = test_data.transpose(0,2,1)

In [30]:
train_data_del_lstm = train_data_del.transpose(0,2,1)
test_data_del_lstm = test_data_del.transpose(0,2,1)

In [36]:
import keras
from keras.models import Model, Sequential
from keras.layers import Dense, Activation, Dropout, LSTM, Input, Permute, TimeDistributed, Bidirectional, BatchNormalization, Flatten, MaxPooling1D, Conv1D, concatenate, Masking, Lambda
from keras.losses import mean_absolute_percentage_error, mean_absolute_error
from sklearn.metrics import mean_squared_error
import keras.backend as K
from sklearn import metrics
from keras_contrib.layers import CRF
from keras import losses
from keras.optimizers import Adam

In [39]:
def create_model_CNNLSTM_1():
    inputs = Input(shape=(64, 1000))
    x = Bidirectional(LSTM(64,return_sequences=True))(inputs)
    x = Conv1D(15, 3, padding='same', activation='relu')(x)
    x = TimeDistributed(Dense(15))(x)
    outputs = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=inputs, outputs=outputs)
    adam = Adam(lr=0.0001)
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    model.summary()
    return model

In [40]:
model = create_model_CNNLSTM_1()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 64, 1000)          0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 64, 128)           545280    
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 64, 15)            5775      
_________________________________________________________________
time_distributed_3 (TimeDist (None, 64, 15)            240       
_________________________________________________________________
dense_6 (Dense)              (None, 64, 1)             16        
Total params: 551,311
Trainable params: 551,311
Non-trainable params: 0
_________________________________________________________________


In [42]:
model.fit(train_data_lstm, np.array(np.expand_dims(train_labels, axis=2)), epochs=5, batch_size=100)


Epoch 1/5
1089/1089 [==============================] - 7s 6ms/step - loss: 0.4121 - acc: 0.8533
Epoch 2/5
1089/1089 [==============================] - 3s 3ms/step - loss: 0.2738 - acc: 0.9265
Epoch 3/5
1089/1089 [==============================] - 3s 3ms/step - loss: 0.2456 - acc: 0.9265
Epoch 4/5
1089/1089 [==============================] - 3s 3ms/step - loss: 0.2373 - acc: 0.9265
Epoch 5/5
1089/1089 [==============================] - 3s 3ms/step - loss: 0.2310 - acc: 0.9265


In [43]:
temp = model.predict(train_data_lstm)

In [112]:
train_data[0]

array([[0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 0, 0, 1],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 1, 0],
       [0, 0, 0, ..., 1, 0, 0]])

In [45]:
temp

array([[[0.05320346],
        [0.05041271],
        [0.05201149],
        ...,
        [0.05396241],
        [0.06884575],
        [0.41918635]],

       [[0.14807227],
        [0.0255613 ],
        [0.03214759],
        ...,
        [0.05584863],
        [0.07527369],
        [0.42017433]],

       [[0.17115894],
        [0.09509912],
        [0.09401262],
        ...,
        [0.05085436],
        [0.06576887],
        [0.42913264]],

       ...,

       [[0.16446817],
        [0.08871534],
        [0.08695814],
        ...,
        [0.03194019],
        [0.05870402],
        [0.42399254]],

       [[0.1694614 ],
        [0.09261608],
        [0.09130216],
        ...,
        [0.02075046],
        [0.03629437],
        [0.39796427]],

       [[0.15002176],
        [0.02286321],
        [0.03297549],
        ...,
        [0.05518556],
        [0.074875  ],
        [0.42348173]]], dtype=float32)

In [48]:
res = []
for i in range(len(train_labels)):
    for j in range(64):
        if train_labels[i][j] == 1:
            res.append(temp[i][j][0])

In [49]:
res

[0.055028677,
 0.29436517,
 0.05277887,
 0.41918635,
 0.2558093,
 0.04871908,
 0.28476214,
 0.1016843,
 0.046128333,
 0.045082748,
 0.053061843,
 0.05411595,
 0.062681854,
 0.09131074,
 0.42017433,
 0.08573946,
 0.08532721,
 0.07719526,
 0.050798327,
 0.16087666,
 0.30463868,
 0.29999322,
 0.3335622,
 0.24812114,
 0.10842425,
 0.057516158,
 0.07287526,
 0.07313451,
 0.07085648,
 0.05128023,
 0.4052777,
 0.28514877,
 0.095956415,
 0.41962823,
 0.08405486,
 0.034938753,
 0.048684657,
 0.040203482,
 0.25660062,
 0.0147512555,
 0.017520398,
 0.15072083,
 0.05073726,
 0.42026025,
 0.07946795,
 0.28793743,
 0.09690431,
 0.42033583,
 0.0209651,
 0.014808089,
 0.07455504,
 0.27360368,
 0.28785235,
 0.054308265,
 0.42459798,
 0.087768495,
 0.09405008,
 0.27054563,
 0.11716786,
 0.056164324,
 0.07241434,
 0.07270759,
 0.07063955,
 0.051573336,
 0.42258516,
 0.1316114,
 0.055473953,
 0.41933066,
 0.085347205,
 0.021977156,
 0.04826489,
 0.052250504,
 0.02096492,
 0.057268858,
 0.06560516,
 0.2814

In [60]:
def create_model_CNNLSTM_2():
    inputs = Input(shape=(1000, 64))
    x = Conv1D(64, 3, padding='same', activation='relu')(inputs)
    x = Bidirectional(LSTM(64,return_sequences=False))(x)
    x = Dense(64, activation='relu')(x)
    outputs = Dense(64, activation='sigmoid')(x)
    model = Model(inputs=inputs, outputs=outputs)
    adam = Adam(lr=0.0001)
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    model.summary()
    return model

In [61]:
model = create_model_CNNLSTM_2()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 1000, 64)          0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 1000, 64)          12352     
_________________________________________________________________
bidirectional_7 (Bidirection (None, 128)               66048     
_________________________________________________________________
dense_10 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_11 (Dense)             (None, 64)                4160      
Total params: 90,816
Trainable params: 90,816
Non-trainable params: 0
_________________________________________________________________


In [62]:
train_data.shape

(1089, 1000, 64)

In [64]:
model.fit(train_data, train_labels, epochs=5, batch_size=100)

Epoch 1/5
1089/1089 [==============================] - 116s 106ms/step - loss: 0.6570 - acc: 0.6326
Epoch 2/5
1089/1089 [==============================] - 129s 119ms/step - loss: 0.4855 - acc: 0.8003
Epoch 3/5
1089/1089 [==============================] - 122s 112ms/step - loss: 0.3062 - acc: 0.8998
Epoch 4/5
1089/1089 [==============================] - 120s 110ms/step - loss: 0.2134 - acc: 0.9184
Epoch 5/5
1089/1089 [==============================] - 116s 106ms/step - loss: 0.1862 - acc: 0.9308


In [66]:
temp2 = model.predict(train_data)

In [106]:
temp2[0]

array([2.54990160e-02, 8.80670846e-02, 1.52132928e-01, 8.55311751e-03,
       2.31365561e-02, 2.62269676e-02, 1.02128088e-02, 1.66684330e-01,
       8.16375017e-04, 5.61956763e-02, 1.53124332e-03, 3.69791090e-02,
       3.03476751e-02, 3.84195149e-02, 4.04268205e-02, 3.82737160e-01,
       3.45785916e-02, 4.51854467e-02, 1.12696588e-02, 2.97486782e-04,
       2.56431103e-03, 1.18703544e-02, 1.06206536e-02, 8.31487179e-02,
       4.68194485e-05, 5.32627106e-04, 1.58489048e-02, 3.38670313e-02,
       1.48156583e-02, 4.30575907e-02, 6.84019923e-03, 3.74315172e-01,
       1.00826293e-01, 1.08558446e-01, 2.05782056e-03, 1.74362063e-02,
       4.38138843e-03, 2.69263983e-03, 1.16535932e-01, 2.67553926e-01,
       1.33091807e-02, 5.29291928e-02, 7.25905299e-02, 6.61502182e-02,
       2.22729743e-02, 5.61107099e-02, 1.11007690e-03, 3.12802434e-01,
       5.13776839e-02, 9.11023319e-02, 8.81306231e-02, 1.28726631e-01,
       1.94875002e-02, 2.68246830e-02, 6.28969073e-03, 2.14756817e-01,
      

In [70]:
res2 = []
for i in range(len(train_labels)):
    for j in range(64):
        if train_labels[i][j] == 1:
            res2.append(temp2[i][j])

In [72]:
res2

[0.38273716,
 0.37431517,
 0.54336447,
 0.62177944,
 0.16894001,
 0.4455287,
 0.09482521,
 0.117473125,
 0.10491675,
 0.30249336,
 0.09082699,
 0.07922655,
 0.23764232,
 0.6189401,
 0.6185725,
 0.034160405,
 0.048228264,
 0.093035966,
 0.05414468,
 0.099793315,
 0.044376552,
 0.4494014,
 0.62437916,
 0.42853475,
 0.054416776,
 0.051221967,
 0.080749124,
 0.12816599,
 0.17020646,
 0.55039036,
 0.63222533,
 0.24038142,
 0.5830214,
 0.6280287,
 0.09035945,
 0.3722129,
 0.04197836,
 0.3610322,
 0.24282777,
 0.3900802,
 0.38080746,
 0.35963118,
 0.52572536,
 0.6191894,
 0.3740034,
 0.33742023,
 0.6238574,
 0.5960424,
 0.41332358,
 0.4108249,
 0.049961984,
 0.27572477,
 0.064126015,
 0.60730743,
 0.577386,
 0.3034463,
 0.1490564,
 0.42738235,
 0.05584261,
 0.052366167,
 0.08032754,
 0.13360244,
 0.17518765,
 0.5444994,
 0.6263412,
 0.06997821,
 0.55604714,
 0.5855902,
 0.039711803,
 0.021391928,
 0.06204936,
 0.42594656,
 0.3662277,
 0.37250218,
 0.058839053,
 0.29527932,
 0.030872822,
 0.04

In [73]:
count = 0
for i in range(len(res2)):
    if res2[i] > 0.5:
        count = count + 1

In [74]:
count, count / len(res2)

(1187, 0.23165495706479314)

In [76]:
def create_model_CNNLSTM_3():
    inputs = Input(shape=(64, 1000))
    x = Masking(mask_value = -1)(inputs)
    x = Bidirectional(LSTM(64,return_sequences=True))(x)
    x = Lambda(lambda x: x, output_shape=lambda s: s)(x)
    x = Conv1D(15, 3, padding='same', activation='relu')(x)
    x = TimeDistributed(Dense(15))(x)
    outputs = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=inputs, outputs=outputs)
    adam = Adam(lr=0.0001)
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    model.summary()
    return model

In [77]:
model = create_model_CNNLSTM_3()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 64, 1000)          0         
_________________________________________________________________
masking_1 (Masking)          (None, 64, 1000)          0         
_________________________________________________________________
bidirectional_8 (Bidirection (None, 64, 128)           545280    
_________________________________________________________________
lambda_1 (Lambda)            (None, 64, 128)           0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 64, 15)            5775      
_________________________________________________________________
time_distributed_6 (TimeDist (None, 64, 15)            240       
_________________________________________________________________
dense_13 (Dense)             (None, 64, 1)             16  

In [79]:
model.fit(train_data_del_lstm, np.array(np.expand_dims(train_labels_del, axis=2)), epochs=5, batch_size=100)

Epoch 1/5
1089/1089 [==============================] - 9s 9ms/step - loss: -2.0012 - acc: 0.3274
Epoch 2/5
1089/1089 [==============================] - 4s 4ms/step - loss: -5.4207 - acc: 0.3405
Epoch 3/5
1089/1089 [==============================] - 4s 4ms/step - loss: -8.5496 - acc: 0.3403
Epoch 4/5
1089/1089 [==============================] - 4s 4ms/step - loss: -9.1224 - acc: 0.3368
Epoch 5/5
1089/1089 [==============================] - 6s 5ms/step - loss: -9.2416 - acc: 0.3179


In [81]:
temp3 = model.predict(train_data_del_lstm)

In [82]:
temp3

array([[[4.2762583e-01],
        [3.2154393e-01],
        [1.4464149e-01],
        ...,
        [0.0000000e+00],
        [0.0000000e+00],
        [2.9802322e-08]],

       [[4.2936775e-01],
        [3.3209598e-01],
        [1.4702287e-01],
        ...,
        [0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00]],

       [[4.2516133e-01],
        [3.2260785e-01],
        [1.4374569e-01],
        ...,
        [0.0000000e+00],
        [0.0000000e+00],
        [8.9406967e-08]],

       ...,

       [[4.2636669e-01],
        [2.9537421e-01],
        [1.7597750e-01],
        ...,
        [0.0000000e+00],
        [0.0000000e+00],
        [8.9406967e-08]],

       [[4.2403859e-01],
        [3.1773108e-01],
        [1.4115372e-01],
        ...,
        [0.0000000e+00],
        [0.0000000e+00],
        [8.9406967e-08]],

       [[4.2864490e-01],
        [3.3022830e-01],
        [1.4401677e-01],
        ...,
        [0.0000000e+00],
        [0.0000000e+00],
        [8.9406967e-08]]

In [83]:
res3 = []
for i in range(len(train_labels)):
    for j in range(64):
        if train_labels[i][j] == 1:
            res3.append(temp3[i][j])

In [84]:
res3

[array([0.10709158], dtype=float32),
 array([0.09799352], dtype=float32),
 array([0.], dtype=float32),
 array([2.9802322e-08], dtype=float32),
 array([0.09086716], dtype=float32),
 array([0.], dtype=float32),
 array([0.], dtype=float32),
 array([0.], dtype=float32),
 array([0.], dtype=float32),
 array([0.], dtype=float32),
 array([0.], dtype=float32),
 array([0.], dtype=float32),
 array([0.], dtype=float32),
 array([0.], dtype=float32),
 array([0.], dtype=float32),
 array([0.], dtype=float32),
 array([0.], dtype=float32),
 array([0.], dtype=float32),
 array([0.], dtype=float32),
 array([0.34860438], dtype=float32),
 array([0.0999831], dtype=float32),
 array([0.], dtype=float32),
 array([8.940697e-08], dtype=float32),
 array([0.], dtype=float32),
 array([0.], dtype=float32),
 array([0.], dtype=float32),
 array([0.], dtype=float32),
 array([0.], dtype=float32),
 array([0.], dtype=float32),
 array([0.], dtype=float32),
 array([8.940697e-08], dtype=float32),
 array([0.], dtype=float32),
 a

In [86]:
count = 0
for i in range(len(res3)):
    if res3[i] > 0.5:
        count = count + 1

In [87]:
count, count / len(res3)

(10, 0.00195160031225605)

In [88]:
def create_model_CNNLSTM_4():
    inputs = Input(shape=(1000, 64))
    x = Masking(mask_value = -1)(inputs)
    x = Lambda(lambda x: x, output_shape=lambda s: s)(x)
    x = Conv1D(64, 3, padding='same', activation='relu')(x)
    x = Bidirectional(LSTM(64,return_sequences=False))(x)
    x = Dense(64, activation='relu')(x)
    outputs = Dense(64, activation='sigmoid')(x)
    model = Model(inputs=inputs, outputs=outputs)
    adam = Adam(lr=0.0001)
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    model.summary()
    return model

In [89]:
model = create_model_CNNLSTM_4()

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 1000, 64)          0         
_________________________________________________________________
masking_2 (Masking)          (None, 1000, 64)          0         
_________________________________________________________________
lambda_2 (Lambda)            (None, 1000, 64)          0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 1000, 64)          12352     
_________________________________________________________________
bidirectional_9 (Bidirection (None, 128)               66048     
_________________________________________________________________
dense_14 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_15 (Dense)             (None, 64)                4160

In [90]:
model.fit(train_data_del, train_labels_del, epochs=5, batch_size=100)

Epoch 1/5
1089/1089 [==============================] - 127s 117ms/step - loss: 0.3048 - acc: 0.2559
Epoch 2/5
1089/1089 [==============================] - 126s 116ms/step - loss: -1.0789 - acc: 0.3052
Epoch 3/5
1089/1089 [==============================] - 124s 113ms/step - loss: -3.7005 - acc: 0.3198
Epoch 4/5
1089/1089 [==============================] - 126s 116ms/step - loss: -7.1053 - acc: 0.3290
Epoch 5/5
1089/1089 [==============================] - 126s 116ms/step - loss: -8.5441 - acc: 0.3405


In [91]:
temp4 = model.predict(train_data_del)

In [105]:
temp4[0]

array([7.0353240e-02, 1.4733896e-01, 6.2693328e-02, 1.4410663e-01,
       1.5903294e-02, 3.0368567e-05, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 2.1561086e-03, 0.0000000e+00, 5.9604645e-08,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 8.0466270e-07,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       1.3709068e-05, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 5.6564808e-04, 0.0000000e+00, 1.1920929e-07,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 1.1026859e-06,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e

In [93]:
res4 = []
for i in range(len(train_labels)):
    for j in range(64):
        if train_labels[i][j] == 1:
            res4.append(temp4[i][j])

In [94]:
res4

[8.046627e-07,
 1.1920929e-07,
 0.0,
 0.0,
 0.0,
 1.4901161e-07,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.14737985,
 0.0,
 0.0,
 0.0,
 8.6426735e-07,
 0.0,
 0.0,
 0.0,
 0.0,
 1.2814999e-06,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.14737883,
 8.046627e-07,
 0.0,
 1.4901161e-07,
 0.0,
 0.0,
 8.6426735e-07,
 1.4901161e-07,
 0.0,
 0.0,
 8.6426735e-07,
 0.0,
 0.0,
 0.0,
 8.6426735e-07,
 1.1920929e-07,
 0.0,
 0.0,
 0.0021559596,
 0.0,
 0.0,
 0.0,
 0.06321302,
 8.046627e-07,
 0.0,
 0.0,
 0.0,
 0.0,
 1.1026859e-06,
 0.0,
 0.0,
 0.0021558404,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 8.6426735e-07,
 1.1920929e-07,
 0.0,
 0.0,
 1.7881393e-07,
 0.0,
 0.0,
 9.23872e-07,
 0.0,
 0.0,
 8.6426735e-07,
 0.0,
 0.0,
 0.0,
 0.0,
 1.1026859e-06,
 0.0,
 0.0,
 0.06321302,
 0.0,
 5.9604645e-08,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 3.0338764e-05,
 0.0,
 0.0,
 0.0,
 0.0,
 0.06321302,
 0.0,
 1.1920929e-07,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 8.64267

In [95]:
count = 0
for i in range(len(res4)):
    if res4[i] > 0.5:
        count = count + 1

In [96]:
count, count / len(res4)

(0, 0.0)

In [97]:
import keras
from keras import backend as K
import tensorflow as tf
from keras.optimizers import Adam

# Define our custom loss function
def focal_loss(y_true, y_pred):
    gamma = 2.0
    alpha = 0.25
    pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
    pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
    return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))

In [98]:
def create_model_CNNLSTM_5():
    inputs = Input(shape=(1000, 64))
    x = Conv1D(64, 3, padding='same', activation='relu')(inputs)
    x = Bidirectional(LSTM(64,return_sequences=False))(x)
    x = Dense(64, activation='relu')(x)
    outputs = Dense(64, activation='sigmoid')(x)
    model = Model(inputs=inputs, outputs=outputs)
    adam = Adam(lr=0.0001)
    model.compile(loss=[focal_loss], metrics=["accuracy"], optimizer=adam)
    model.summary()
    return model

In [99]:
model.fit(train_data, train_labels, epochs=5, batch_size=100)

Epoch 1/5
1089/1089 [==============================] - 125s 115ms/step - loss: 1.0943 - acc: 0.9265
Epoch 2/5
1089/1089 [==============================] - 124s 114ms/step - loss: 1.0610 - acc: 0.9265
Epoch 3/5
1089/1089 [==============================] - 119s 109ms/step - loss: 0.9876 - acc: 0.9264
Epoch 4/5
1089/1089 [==============================] - 120s 111ms/step - loss: 0.4615 - acc: 0.9175
Epoch 5/5
1089/1089 [==============================] - 118s 108ms/step - loss: 0.2595 - acc: 0.9234


In [100]:
temp5 = model.predict(train_data)

In [101]:
res5 = []
for i in range(len(train_labels)):
    for j in range(64):
        if train_labels[i][j] == 1:
            res5.append(temp5[i][j])

In [102]:
res5

[0.5868388,
 0.43893442,
 0.087784946,
 0.11196047,
 0.06701478,
 0.43414742,
 0.06709111,
 0.09455514,
 0.050827354,
 0.13282111,
 0.047981054,
 0.057066888,
 0.14149505,
 0.08659673,
 0.107699215,
 0.026343226,
 0.05026692,
 0.039226174,
 0.023565143,
 0.23708269,
 0.018761724,
 0.09850642,
 0.077394485,
 0.59115976,
 0.091784894,
 0.07246351,
 0.1380504,
 0.15485165,
 0.14262232,
 0.14784536,
 0.15108892,
 0.11173418,
 0.07477087,
 0.0904184,
 0.2533214,
 0.57849884,
 0.07006788,
 0.39401788,
 0.11738008,
 0.10711664,
 0.5804476,
 0.45153013,
 0.10200101,
 0.12433857,
 0.6071234,
 0.10549313,
 0.068879575,
 0.084579855,
 0.61580175,
 0.38101795,
 0.02731809,
 0.09069574,
 0.24994409,
 0.107004404,
 0.12643692,
 0.10908818,
 0.17849877,
 0.5919224,
 0.08436844,
 0.07036027,
 0.13207638,
 0.1459234,
 0.13941222,
 0.13802496,
 0.14291406,
 0.25911003,
 0.1197097,
 0.13599673,
 0.09877738,
 0.18291971,
 0.08770555,
 0.22260275,
 0.5973556,
 0.38469616,
 0.032014817,
 0.09816775,
 0.0798

In [103]:
count = 0
for i in range(len(res5)):
    if res5[i] > 0.5:
        count = count + 1

In [104]:
count, count / len(res5)

(436, 0.08508977361436378)

In [93]:
labels = []
preds = []
for i in range(len(train_labels)):
    for j in range(64):
        if train_labels[i][j] == 1:
            res = [np.argmax(t) for t in temp[i]]
            preds.append(res[j])

In [94]:
preds.count(1),len(preds),preds.count(1)/len(preds)

(89, 5124, 0.017369242779078843)

In [99]:
def cal_score(model, train_data, train_labels):
    print("数据集:")
    predict_target_msb_prob = model.predict(train_data)
    predict_target_msb_label = (predict_target_msb_prob > 0.5).astype(int)
    predict_target_msb_1D = predict_target_msb_label.flatten()
    train_labels_1D = train_labels.flatten()
    print("预测正确数量,训练集样本量:")
    print(sum(predict_target_msb_1D == train_labels_1D),len(train_labels_1D))
    print("训练集精确度等指标：")
    print(metrics.classification_report(train_labels_1D,predict_target_msb_1D))
    print("混淆矩阵：")
    print(metrics.confusion_matrix(train_labels_1D,predict_target_msb_1D))